# TensorFlow Custom Object Detection

### `Important Links`
__`TensorFLow Models`__ : https://github.com/tensorflow/models

__`PreTrained Model`__ : 
http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz

__`Protocol Buffers`__ : https://github.com/protocolbuffers/protobuf/releases/tag/v3.19.4

## Step 1: Create Folder Structure

In [ ]:
import os
import wget

In [ ]:
custom_model_name = 'batman_detector' 
pretrained_model = 'ssd_mobilenet_v2_320x320_coco17_tpu-8'
tf_record_script = 'generate_tfrecord.py'
label_map = 'label_map.pbtxt'

In [ ]:
paths = {
    'tensorflow_path': os.path.join('Tensorflow'),    
    'workspace_path': os.path.join('Tensorflow', 'workspace'),
    'scripts_path': os.path.join('Tensorflow','scripts'),
    'api_path': os.path.join('Tensorflow','models'),
    'annotation_path': os.path.join('Tensorflow', 'workspace','annotations'),
    'image_path': os.path.join('Tensorflow', 'workspace','images'),
    'train_image_path': os.path.join('Tensorflow', 'workspace','images','train'),
    'test_image_path': os.path.join('Tensorflow', 'workspace','images','test'),
    'model_path': os.path.join('Tensorflow', 'workspace','models'),
    'pretrained_model_path': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'checkpoint_path': os.path.join('Tensorflow', 'workspace','models', custom_model_name), 
    'output_path': os.path.join('Tensorflow', 'workspace','models', custom_model_name, 'export'), 
    'protoc_path':os.path.join('Tensorflow','protoc')
 }

In [ ]:
files = {
    'pipeline_config':os.path.join('Tensorflow', 'workspace','models', custom_model_name, 'pipeline.config'),
    'tf_record_script': os.path.join(paths['scripts_path'], tf_record_script), 
    'labelmap': os.path.join(paths['annotation_path'], label_map)
}

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        os.mkdir(path)

## Step 2: Installing TensorFlow Object Detection API

In [ ]:
download_urls = {
    'protobuff': "https://github.com/protocolbuffers/protobuf/releases/download/v3.19.4/protoc-3.19.4-win64.zip",
    'tensorflow_models': "https://github.com/tensorflow/models/archive/refs/heads/master.zip",
    'pretrained_model': "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz"
}

In [ ]:
# Download Protoc & Extract files
wget.download(download_urls["protobuff"], paths['protoc_path'])
!cd {paths['protoc_path']} && tar -xf protoc-3.19.4-win64.zip

In [ ]:
# Add protoc/bin to environment path
os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['protoc_path'], 'bin')) 

In [ ]:
# Download & Extract TensorFlow Object Detection API
wget.download(download_urls["tensorflow_models"], paths['api_path'])
!cd {paths['api_path']} && tar -xf models-master.zip --strip-components 1

In [ ]:
# Download & Extract Pretrained Model from TensorFlow Zoo
wget.download(download_urls["pretrained_model"], paths['pretrained_model_path'])
!cd {paths['pretrained_model_path']} && tar -zxvf {pretrained_model + '.tar.gz'}

In [ ]:
tfrecord_url = "https://github.com/afaqueumer/TensorFlow-Object-Detection/raw/main/generate_tfrecord.zip"
wget.download(tfrecord_url, paths['scripts_path'])
!cd {paths['scripts_path']} && tar -xf generate_tfrecord.zip

In [ ]:
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
!cd Tensorflow/models/research/slim && pip install -e 
verification_script = os.path.join(paths['api_path'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
!python {verification_script}

In [ ]:
import object_detection

## Step 3: Preparing the Dataset

In [ ]:
labels = [{'id':1, 'name':'Batman'}]
with open(files['labelmap'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write(f'\tid:{label["id"]}\n')
        f.write(f'\tname:\'{label["name"]}\'\n')
        f.write('}\n')

In [ ]:
!python {files['tf_record_script']} -x {os.path.join(paths['image_path'], 'train')} -l {files['labelmap']} -o {os.path.join(paths['annotation_path'], 'train.record')} 
!python {files['tf_record_script']} -x {os.path.join(paths['image_path'], 'test')} -l {files['labelmap']} -o {os.path.join(paths['annotation_path'], 'test.record')} 

## Step 4: Configuring the Model

In [ ]:
!copy {os.path.join(paths['pretrained_model_path'], pretrained_model, 'pipeline.config')} {os.path.join(paths['checkpoint_path'])}

Modify the above pipeline.config file with paths and parameters.

## Step 5: Training the model

In [ ]:
!copy {os.path.join(paths['api_path'], 'research', 'object_detection', 'model_main_tf2.py')} {paths['scripts_path']}

In [ ]:
training_script = os.path.join(paths['scripts_path'], 'model_main_tf2.py')
training_script

In [ ]:
training_command = "python {} --model_dir={} --pipeline_config_path={}".format(training_script, paths['checkpoint_path'],files['pipeline_config'])

In [ ]:
print(training_command)

In [ ]:
# Train the Model
#!{training_command}

## Step 6: Exporting the Model and Making Detections

In [ ]:
!copy {os.path.join('TensorFlow','models', 'research', 'object_detection', 'exporter_main_v2.py')} {paths['scripts_path']}

In [ ]:
exporting_script = os.path.join(paths['scripts_path'], 'exporter_main_v2.py')
exporting_script

In [ ]:
exporting_command = "python {} --input_type image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(exporting_script, files['pipeline_config'], paths['checkpoint_path'], paths['output_path'])

In [ ]:
print(exporting_command)

In [ ]:
# Export the Model
# !{exporting_command}

In [ ]:
import time
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
%matplotlib inline
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import tensorflow as tf
import numpy as np
%matplotlib inline

# Provide testing image path
testing_image_path = os.path.join('Tensorflow' , 'workspace', 'images', 'batman', 'jj.jpg')
model_dir = paths["checkpoint_path"]
labels_path = files["labelmap"] 
saved_model_path = os.path.join(model_dir ,"export" , "saved_model")

# Load exported model
print('Loading model...', end='')
start_time = time.time()
detect_fn = tf.saved_model.load(saved_model_path)
end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# Begin Detection
category_index = label_map_util.create_category_index_from_labelmap(labels_path, use_display_name=True)
image_np = np.array(Image.open(testing_image_path))
input_tensor = tf.convert_to_tensor(image_np)
input_tensor = input_tensor[tf.newaxis, ...]
detections = detect_fn(input_tensor)
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
detections['num_detections'] = num_detections
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
image_np_with_detections = image_np.copy()
viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=1,
          min_score_thresh=.2,
          agnostic_mode=False)

# Display the image with detected box
plt.imshow(image_np_with_detections)